## Comparación de modelos COCO Person Keypoint Detection Baselines with Keypoint R-CNN 
### Autor: José Miguel Ramírez Sanz
#### Imports

In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import timeit
import math
import os
import pandas as pd
import pickle as pk
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

# import de utilities de detectron
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

#### Modelos

In [2]:
COCOPerson=["COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml",
        "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml",
        "COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml",
        "COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"]

#### Carga de los vídeos

In [3]:
vds = ['../pruebas/videos-prueba/Videos/Josemi/sentado3-caja.webm',
       '../pruebas/videos-prueba/Videos/Josemi/depie.webm',
      '../pruebas/videos-prueba/Videos/Josemi/sentado1.webm',
      '../pruebas/videos-prueba/Videos/Josemi/sentado2-cruzado.webm',
      '../pruebas/videos-prueba/Videos/Josemi/sentado4-remangado.webm',
      '../pruebas/videos-prueba/Videos/Josemi/sentado5-chaqueta-abierta.webm',
      '../pruebas/videos-prueba/Videos/Josemi/sentado6-camiseta.webm']

#### Función

In [8]:
def posicionamiento(modelo,vd,th,vis=False):
    
    #Tiempo de carga del modelo
    start1 = timeit.default_timer()
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(modelo))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = th
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(modelo)
    predictor = DefaultPredictor(cfg)
    
    stop1 = timeit.default_timer()
    
    vc = cv2.VideoCapture(vd)

    if (vc.isOpened()==False):
        print("Error")
    
    #Tiempo de procesado del vídeo
    start = timeit.default_timer()
    (grabbed, frame) = vc.read()
    i=0
    while(vc.isOpened()):
        i+=1
        ret,frame = vc.read()
        if ret == True:
            o = predictor(frame)
            if vis:
                v = Visualizer(frame[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
                v = v.draw_instance_predictions(o["instances"].to("cpu"))
                imVisualizer = cv2.cvtColor(v.get_image()[:, :, ::-1],cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(18, 16))
                plt.imshow(imVisualizer)
                plt.title("Frame" + str(i))
                plt.show()
        else:
            break
        print("Frame " + str(i),end="\r")
    vc.release()
    cv2.destroyAllWindows()
    
    stop = timeit.default_timer()
    
    return [modelo,stop1-start1,stop-start,i-1]

In [18]:
res=[]
for vd in vds:
    print(vd)
    for i in COCOPerson:
        print(i)
        res.append(posicionamiento(i,vd,0.99))
    print("\n")

../pruebas/videos-prueba/Videos/Josemi/sentado3-caja.webm
COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml
COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml
Frame 300

../pruebas/videos-prueba/Videos/Josemi/depie.webm
COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml
COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml
Frame 259

../pruebas/videos-prueba/Videos/Josemi/sentado1.webm
COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml
COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml
Frame 446

../pruebas/videos-prueba/Videos/Josemi/sentado2-cruzado.webm
COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml
COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml
COCO-Keypoints/keypoint_rcnn_X

In [20]:
a = pd.DataFrame(res)

In [23]:
a=a.rename(columns={0:"Modelo",1:"TCarga",2:"TProc",3:"NFrames"})

In [24]:
a.head()

,Modelo,TCarga,TProc,NFrames
0,COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml,0.982418,32.405615,300
1,COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml,0.967697,32.348420,300
2,COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml,1.341485,40.043822,300
3,COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3...,2.001302,61.341547,300
4,COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml,0.942516,27.325322,259


In [26]:
suma = a.groupby('Modelo').sum()

In [27]:
suma["Ratio"]=suma["TProc"]/suma["NFrames"]

In [28]:
suma

,TCarga,TProc,NFrames,Ratio
Modelo,,,,
COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml,13.017239,279.327051,1989,0.140436
COCO-Keypoints/keypoint_rcnn_R_50_FPN_1x.yaml,8.562254,237.289880,1989,0.119301
COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml,9.783479,236.774504,1989,0.119042
COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml,18.106832,422.024270,1989,0.212179


In [29]:
print(suma.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &     TCarga &       TProc &  NFrames &     Ratio \\
Modelo                                             &            &             &          &           \\
\midrule
COCO-Keypoints/keypoint\_rcnn\_R\_101\_FPN\_3x.yaml     &  13.017239 &  279.327051 &     1989 &  0.140436 \\
COCO-Keypoints/keypoint\_rcnn\_R\_50\_FPN\_1x.yaml      &   8.562254 &  237.289880 &     1989 &  0.119301 \\
COCO-Keypoints/keypoint\_rcnn\_R\_50\_FPN\_3x.yaml      &   9.783479 &  236.774504 &     1989 &  0.119042 \\
COCO-Keypoints/keypoint\_rcnn\_X\_101\_32x8d\_FPN\_3x... &  18.106832 &  422.024270 &     1989 &  0.212179 \\
\bottomrule
\end{tabular}

